In [1]:
import sys
sys.path.append('../')

import time

from  models import utils, Simple_CNN

import matplotlib.pyplot as plt

import torch
from torchvision import transforms
import torchaudio

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split

from torch.utils.data import DataLoader

from sklearn.preprocessing import OneHotEncoder


import numpy as np

import yaml

import pandas as pd

In [2]:
random_seed = 42
net_name = "FlattenCNN-1-4"


with open("./yamls/FlattenCNN.yaml", "r") as f:
    net_archi = yaml.load(f, Loader=yaml.FullLoader)

batch_size = 16
lr = net_archi[net_name]["lr"]
epochs = net_archi[net_name]["epochs"]


# Dataset transformations
transform_img = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)),
     torchaudio.transforms.FrequencyMasking(freq_mask_param=40),
     torchaudio.transforms.TimeMasking(time_mask_param=40),
    ]
)

# Create dataset
dataset = utils.SpectrogramDataset("../datasets/full_dataset_df.csv",
                                   "../spectrograms/full_dataset",
                                   "gbifID", "family",
                                   transform=transform_img,
                                   one_hot_encode_labels=True,
                                   preload=False)

test_percentage = .2
test_len = int((len(dataset) * test_percentage))
train_len = len(dataset) - test_len

train, test = random_split(dataset, [train_len, test_len], torch.Generator().manual_seed(random_seed))

train_loader = DataLoader(train, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

len(dataset)

16168

In [3]:
if torch.cuda.device_count():
  model = torch.load("./networks/FlattenCNN-1-4.pth", map_location=torch.device('cuda:0'))
else:
  model = torch.load("./networks/FlattenCNN-1-4.pth", map_location=torch.device('cpu'))

In [4]:
model.eval()

SimpleCNN(
  (conv): ModuleList(
    (0): Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1), padding=same)
    (1): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=same)
    (2): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1), padding=same)
    (3): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1), padding=same)
    (4): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), padding=same)
    (5): Conv2d(256, 512, kernel_size=(2, 2), stride=(1, 1), padding=same)
  )
  (bnorm): ModuleList(
    (0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): BatchNorm2d(512, eps=1e-05, momentum=0.1, 

In [5]:
df = dataset.csv
df.head()

,gbifID,identifier,species,genus,family,class,phylum,file_name
0,1572324720,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
1,1572324719,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
2,1572324718,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
3,1572324717,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
4,1572324716,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...


In [6]:
df[df["class"]=="Reptilia"].shape

(19, 8)

In [7]:
label_dict = {}

In [8]:
arach_index = df[df["class"]=="Arachnida"].index
arach = []

for index in arach_index:
  arach.append(np.argmax(dataset[index]["label"]))

label_dict["Arachnida"] = list(set(arach))

In [9]:
insect_index = df[df["class"]=="Insecta"].index
insect = []

for index in insect_index:
  insect.append(np.argmax(dataset[index]["label"]))

label_dict["Insecta"] = list(set(insect))

In [10]:
acti_index = df[df["class"]=="Actinopterygii"].index
acti = []

for index in acti_index:
  acti.append(np.argmax(dataset[index]["label"]))

label_dict["Actinopterygii"] = list(set(acti))

In [11]:
amphi_index = df[df["class"]=="Amphibia"].index
amphi = []

for index in amphi_index:
  amphi.append(np.argmax(dataset[index]["label"]))

label_dict["Amphibia"] = list(set(amphi))

In [12]:
aves_index = df[df["class"]=="Aves"].index
aves = []

for index in aves_index:
  aves.append(np.argmax(dataset[index]["label"]))

label_dict["Aves"] = list(set(aves))

In [13]:
mamma_index = df[df["class"]=="Mammalia"].index
mamma = []

for index in mamma_index:
  mamma.append(np.argmax(dataset[index]["label"]))

label_dict["Mammalia"] = list(set(mamma))

In [14]:
repti_index = df[df["class"]=="Reptilia"].index
repti = []

for index in repti_index:
  repti.append(np.argmax(dataset[index]["label"]))

label_dict["Reptilia"] = list(set(repti))

In [15]:
label_dict.keys()

dict_keys(['Arachnida', 'Insecta', 'Actinopterygii', 'Amphibia', 'Aves', 'Mammalia', 'Reptilia'])

In [16]:
pos_dict = {
  "Arachnida": 0,
  "Insecta": 1,
  "Actinopterygii": 2,
  "Amphibia": 3,
  "Aves": 4,
  "Mammalia": 5,
  "Reptilia": 6
}

In [17]:
confusion_matrix = {
  "Arachnida": [0, 0, 0, 0, 0, 0, 0],
  "Insecta": [0, 0, 0, 0, 0, 0, 0],
  "Actinopterygii": [0, 0, 0, 0, 0, 0, 0],
  "Amphibia": [0, 0, 0, 0, 0, 0, 0],
  "Aves": [0, 0, 0, 0, 0, 0, 0],
  "Mammalia": [0, 0, 0, 0, 0, 0, 0],
  "Reptilia": [0, 0, 0, 0, 0, 0, 0]
}

In [18]:
torch.cuda.empty_cache()
for x in test_loader:
  
  img = utils.get_cuda(x["image"])
  
  out = model(img)
  
  out = utils.get_numpy(out)
  
  for label, predicted in zip(x["label"], out):
    value = np.argmax(label)
    pred_value = np.argmax(predicted)
    for key, array in label_dict.items():
      if value in array:
        true_key = key
      if pred_value in array:
        predicted_class = pos_dict[key]
    confusion_matrix[true_key][predicted_class] += 1

In [19]:
confusion_matrix

{'Arachnida': [0, 0, 0, 0, 0, 0, 0],
 'Insecta': [0, 13, 0, 0, 17, 3, 0],
 'Actinopterygii': [0, 0, 0, 0, 0, 0, 0],
 'Amphibia': [0, 0, 0, 0, 13, 2, 0],
 'Aves': [0, 5, 0, 0, 2554, 189, 0],
 'Mammalia': [0, 5, 0, 0, 120, 308, 0],
 'Reptilia': [0, 0, 0, 0, 0, 4, 0]}